# Inhomogeneous implicit derivative

$$
\nabla_{\boldsymbol{\Theta}} \tilde{\mathbf{X}}^*_{\boldsymbol{\Theta}} \in \mathbb{R}^{N_D\times 3N}
$$

In [1]:
# Standard libraries
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt

# Package imports
from lammps_implicit_der import SNAP, LammpsImplicitDer
from lammps_implicit_der.systems import BCC, BCC_BINARY, BCC_BINARY_VACANCY, BCC_VACANCY, \
                                        BCC_SIA, FromData, HCP, \
                                        BccVacancyConcentration
from lammps_implicit_der.tools import plot_tools

# No parallel MPI runs in the notebook, but keep comm for consistency
comm = None

# For parallel runs, use the following:
# from lammps_implicit_der.tools import mpi_print, initialize_mpi
# comm, rank = initialize_mpi()

## Create an atomic system. For this example, we will use one vacancy in BCC lattice

### Non-perturbed: $\mathbf{\Theta}(\lambda,m) = \bar{\mathbf{\Theta}}$ potential

Minimize with fixed cell (`minimize=True` and `fix_box_relax=False`).

By definition, the implicit derivative applies in the case of a local minimum (or maximum), therefore, the position minimization is crucial.

In [6]:
alat = 3.16316
ncell_x = 2
vac0 = BCC_VACANCY(alat=alat, ncell_x=ncell_x,
                   snapcoeff_filename='W_REF.snapcoeff',
                   minimize=True, fix_box_relax=False,
                   logname='vac0.log', comm=comm)

Theta0 = vac0.pot.Theta_dict['W']['Theta'].copy()
X_coord0 = vac0.X_coord.copy()
energy0 = vac0.energy


--------------------------------------------------------------------------------
Running LAMMPS with the following arguments:
-screen none -log vac0.log

Setting SNAP potential

                  SNAP coefficients for: W
                          quadraticflag: 0
 Number of parameters (excluding beta0): 55
                                Element:  W  |  R =  0.5000 w =  1.0000

Minimizing energy with the following parameters:
ftol: 1e-08, maxiter: 1000, maxeval: 1000, algo: cg, fix_box_relax: False 

Minimization finished in 4 steps
Initial fmax: 3.107e-01, final fmax: 2.754e-10
Initial fnorm: 1.522e+00, final fnorm: 1.349e-09
Number of atoms: 15, largest force value: 2.754e-10, force norm: 1.349e-09


## Compute the implicit derivative (`self.implicit_derivative()` method)

* `implicit_derivative()` - implicit derivative wrapper, `method` keyword selects the derivative method, options are:

    * `inverse` - Moore-Penrose inverse of the Hessian matrix, `np.linalg.pinv()`
    * `dense` -  pseudoinverse from the system of linear equations, `np.linalg.solve(hessian, mixed_hessian)`
    * `sparse` - sparse linear method, solve for each potential parameter $\Theta_l$ separately
    * `energy` - find impl. der. from the LAMMPS minimization with additional force and energy terms that correspond to a given parameter $\Theta_l$

### Compute the implicit derivative with the dense pseudoinverse Hessian method (`method=dense`)

In [7]:
dX_dTheta_dense = vac0.implicit_derivative(method='dense')

Computing the Hessian...


Hessian (full): 100%|██████████| 45/45 [00:00<00:00, 54.60it/s]

Computing dX_dTheta with linalg.solve, Hessian shape: (45, 45)


### Create a perturbed system, BCC Vacancy with potential:

$$
\mathbf{\Theta}(\lambda,m) = \bar{\mathbf{\Theta}} + \lambda( \mathbf{\Theta}_m -\bar{\mathbf{\Theta}}),
$$
where $m$ is potential sample from `Theta_ens` ensemble and $\lambda$ is perturbation strength.

In [12]:
with open('Theta_ens.pkl', 'rb') as f:
    Theta_ens = pickle.load(f)

delta = 40.0
sample = 5
Theta_perturb = Theta_ens['Theta_mean'] + delta * (Theta_ens['Theta_ens_list'][sample] - Theta_ens['Theta_mean'])

# Save the perturbed Theta to a file
pot = SNAP.from_files(snapcoeff_filename='W_REF.snapcoeff', snapparam_filename='W_REF.snapparam')
pot.Theta_dict['W']['Theta'] = Theta_perturb.copy()
pot.to_files(snapcoeff_filename='W_perturb.snapcoeff', snapparam_filename='W_perturb.snapparam',
             overwrite=True, verbose=True)

# Since the potential is written to the current folder, we specify  data_path='.'
vac_perturb = BCC_VACANCY(alat=alat, ncell_x=ncell_x,
                          data_path='.',
                          snapcoeff_filename='W_perturb.snapcoeff',
                          minimize=True, fix_box_relax=False,
                          logname='vac0.log', comm=comm)

# True minimized positions
X_true = vac_perturb.X_coord.copy()
# True difference in positions with minimum image convention applied
dX_true = vac0.minimum_image(X_true - X_coord0)

Overwriting ./W_perturb.snapcoeff
Saved SNAP coefficients to ./W_perturb.snapcoeff
Saved SNAP parameters to ./W_perturb.snapparam

--------------------------------------------------------------------------------
Running LAMMPS with the following arguments:
-screen none -log vac0.log

Setting SNAP potential

                  SNAP coefficients for: W
                          quadraticflag: 0
 Number of parameters (excluding beta0): 55
                                Element:  W  |  R =  0.5000 w =  1.0000

Minimizing energy with the following parameters:
ftol: 1e-08, maxiter: 1000, maxeval: 1000, algo: cg, fix_box_relax: False 

Minimization finished in 4 steps
Initial fmax: 5.462e-01, final fmax: 2.345e-13
Initial fnorm: 2.676e+00, final fnorm: 1.091e-12
Number of atoms: 15, largest force value: 2.345e-13, force norm: 1.091e-12


## Predict the change in positions with 